# MC_Issues framework demo

In [1]:
from issues.test_issues import IntIssue, StringIssue
from random import choice
from string import ascii_uppercase

## use metric_type.calculate_all(payload) get all issues found with a payload

In [2]:
refs = []
for i in range(10):
    s = ''.join(choice(ascii_uppercase) for i in range(choice(list(range(5,15)))))
    r = StringIssue.calculate_all(s)
    refs.append(r)
    if len(r) != 0:
        print("\n".join(r_["template"] for r_ in r))
    else:
        print(f'No issues for {s}')
    print("#"*10)

WJWXUKPQ is too short
WJWXUKPQ contains duplicate letters
WJWXUKPQ does not contain the letter 'a'
##########
XRLGWCFXHPO is too long
XRLGWCFXHPO contains duplicate letters
XRLGWCFXHPO does not contain the letter 'a'
##########
SMZFYHS is too short
SMZFYHS contains duplicate letters
SMZFYHS does not contain the letter 'a'
##########
TJUHPMOFROGA is too long
TJUHPMOFROGA contains duplicate letters
##########
DLLRPLKFJOZHR is too long
DLLRPLKFJOZHR contains duplicate letters
DLLRPLKFJOZHR does not contain the letter 'a'
##########
OAPHXE is too short
##########
QQGUTWZ is too short
QQGUTWZ contains duplicate letters
QQGUTWZ does not contain the letter 'a'
##########
OHTRTTYGW is too short
OHTRTTYGW contains duplicate letters
OHTRTTYGW does not contain the letter 'a'
##########
JXAQMODCKTJPB is too long
JXAQMODCKTJPB contains duplicate letters
##########
XWMTLEDA is too short
##########


## Use include_tags to only run on certain kinds of issue

In [3]:
for i in range(10):
    s = ''.join(choice(ascii_uppercase) for i in range(choice(list(range(5,15)))))
    r = StringIssue.calculate_all(s, include_tags=["letter"])
    if len(r) != 0:
        print("\n".join(r_["template"] for r_ in r))
    else:
        print(f'No issues for {s}')
    print("#"*10)

No issues for AFLKJ
##########
LUKWYMRUAQGAYW contains duplicate letters
##########
WFFWYCCTBW contains duplicate letters
WFFWYCCTBW does not contain the letter 'a'
##########
GINIJHEHMFFZ contains duplicate letters
GINIJHEHMFFZ does not contain the letter 'a'
##########
IMYTIPDSMJOPGS contains duplicate letters
IMYTIPDSMJOPGS does not contain the letter 'a'
##########
YGLNSHKLDSWQ contains duplicate letters
YGLNSHKLDSWQ does not contain the letter 'a'
##########
MGBNWY does not contain the letter 'a'
##########
YOVOTYHQ contains duplicate letters
YOVOTYHQ does not contain the letter 'a'
##########
NARFBTVPVSWKF contains duplicate letters
##########
OTVXPOJLWUOT contains duplicate letters
OTVXPOJLWUOT does not contain the letter 'a'
##########


## And use exclude_tags to limit what kinds of issues you're checking for

In [4]:
for i in range(10):
    s = ''.join(choice(ascii_uppercase) for i in range(choice(list(range(5,15)))))
    r = StringIssue.calculate_all(s, exclude_tags=["letter"])
    if len(r) != 0:
        print("\n".join(r_["template"] for r_ in r))
    else:
        print(f'No issues for {s}')
    print("#"*10)

JLQTEYAW is too short
##########
NNWUBUDOXWIJ is too long
##########
INEJQRFLKKR is too long
##########
ANOHMBHH is too short
##########
HLTOBFCPONDH is too long
##########
TUSQCI is too short
##########
EGSRG is too short
##########
CQIRTRJEXZZDB is too long
##########
IDIJWS is too short
##########
KOLYFUNOGLFVS is too long
##########


## The framework tries to catch and report errors in issue calculation gracefully
Note that execution isn't halted- we just log an error, and report that the issue didn't calculate right. 

In [15]:
print(IntIssue.calculate_all(2, exclude_tags=["breaking"]))
print(IntIssue.calculate_all(3))
print(IntIssue.calculate_all(8, exclude_tags=["breaking"]))

ERROR:issues:Error calculating issue broken_issue for payload 3: Intentional Error
Traceback (most recent call last):
  File "/Users/pgulley/Projects/directory-issues/issues/__init__.py", line 62, in calculate_all
    is_issue, result_data = issue_instance.calculate(payload)
                            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/pgulley/Projects/directory-issues/issues/test_issues.py", line 63, in calculate
    raise RuntimeError("Intentional Error")
RuntimeError: Intentional Error


[{'issue_name': 'is_even', 'template': '2 is even', 'tags': []}]
[{'issue_name': 'broken_issue', 'tags': ['breaking'], 'error': True, 'error_message': 'Intentional Error', 'template': "An error occurred while calculating 'broken_issue'"}]
[{'issue_name': 'is_even', 'template': '8 is even', 'tags': []}]


# Usage:

We shouldn't need very many extensions on the base issue class- we'll certainly need a SourceIssue, FeedIssue, and a CollectionIssue but that's probably it. 
Regardless, generating one is a two-liner: 

In [6]:
from issues import IssueBase
from typing import TypeVar, Tuple, Generic, Type, Dict, Callable, List, Any

class FloatIssue(IssueBase[float]):
    _ISSUES: Dict[str, Type["FloatIssue"]] = {}


Then actually implimenting the issue cases requires creating a class with just two methods- `calculate(payload)` and `render_template()`, then decorating it correctly:

In [7]:
@FloatIssue.register("decimal_places", tags=['decimal'])
class DecimalPlaces(FloatIssue):
    def calculate(self, payload:float) -> Tuple[bool, Dict]:        
        """
        NB the return signature: Tuple[bool, Any]
        bool is ultimately interpreted as "is_issue" - if false, the result is skipped
        Dict is the input provided to the render template
        """
        num, dec = str(payload).split(".")
        exp = len(dec)
    
        return exp > 2, {'value':payload, "exp":exp}

    def render_template(self):
        return f"{self.result['value']} has too many decimal places ({self.result['exp']})"


@FloatIssue.register("negative")
class NegativeFloat(FloatIssue):
    def calculate(self, payload:float):
        return payload < 0, {'value':payload}

    def render_template(self):
        return f"{self.result['value']} is negative"
            

In [8]:
for i in (1.2, -4.7 ,1.24525):
    r = FloatIssue.calculate_all(i)
    print(r)

[]
[{'issue_name': 'negative', 'template': '-4.7 is negative', 'tags': []}]
[{'issue_name': 'decimal_places', 'template': '1.24525 has too many decimal places (5)', 'tags': ['decimal']}]


At dev time, you should only be calling an issue from the calculate_all method- there's no need to ever call issue.calculate or issue.render_template directly. 


In [9]:
print(refs[3])

[{'issue_name': 'long_str', 'template': 'TJUHPMOFROGA is too long', 'tags': []}, {'issue_name': 'dup_letter', 'template': 'TJUHPMOFROGA contains duplicate letters', 'tags': ['letter']}]
